<a href="https://colab.research.google.com/github/fastai-energetic-engineering/ashrae/blob/master/parquet_to_tabular_data_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 30.9MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 1.2MB 44.4MB/s 
     |████████████████████████████████| 194kB 54.4MB/s 
     |████████████████████████████████| 61kB 10.4MB/s 
     |████████████████████████████████| 61kB 10.6MB/s 
Mounted at /content/gdrive


In [2]:
#collapse
import datetime
import os

import pandas as pd
from fastai.tabular.all import *
from fastbook import *
from tqdm.auto import tqdm

In [3]:
tqdm.pandas(desc="apply progress")

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
%cd /content/gdrive/MyDrive/Colab Notebooks/ashrae/

/content/gdrive/MyDrive/Colab Notebooks/ashrae


In [5]:
df = pd.read_parquet("train_combined.parquet.snappy")

In [6]:
year_fraction = df["timestamp"] - df["timestamp"].iloc[0]

In [7]:
df["year_fraction"] = year_fraction / year_fraction.max()

In [8]:
df["week_fraction"] = df["timestamp"].progress_apply(lambda ts: (ts - pd.Timestamp((ts - pd.Timedelta(days=ts.dayofweek)).date())) / pd.Timedelta(days=7))

In [9]:
df["day_fraction"] = df["timestamp"].progress_apply(lambda ts: (ts - pd.Timestamp(ts.date())) / pd.Timedelta(hours=24))

In [10]:
# Save or load the feature engineered dataframe here

df.to_parquet("feature_enhanced_train_combined.parquet.snappy")
# df = pd.read_parquet("feature_enhanced_train_combined.parquet.snappy")

In [ ]:
cat_names = ["building_id", "site_id", "primary_use"]
cont_names = [
    "year_fraction",
    "week_fraction",
    "day_fraction"
    "square_feet",
    "year_built",
    "floor_count",
    "air_temperature",
    "cloud_coverage",
    "dew_temperature",
    "precip_depth_1_hr",
    "sea_level_pressure",
    "wind_direction",
    "wind_speed",
]
y_names = ["meter_0", "meter_1", "meter_2", "meter_3"]
procs = [Categorify, FillMissing, Normalize]

In [ ]:
dls = TabularDataLoaders.from_df(
    df=df,
    procs=procs,
    cat_names=cat_names,
    cont_names=cont_names,
    y_names=y_names,
    shuffle=False
)

In [ ]:
dls.show_batch()

In [ ]:
# Free up some RAM by deallocating the DataFrame
del df